# CS 5683 - Big Data Analytics
## Project - 1: A priori algorithm
### Suraj Pawar

In [3]:
!pip install pyspark

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/3wE9Nv6pC2TC2_bATbsvj3m9HS1NiSWaZ3t4Wa-fn2W49Rn0Gwg5uAY
Mounted at /content/drive


In [5]:
import os
import shutil
import sys
from operator import add
import itertools

from pyspark import SparkContext

DEBUG = 1

sc = SparkContext(appName="Spark Apriori")


In [6]:
import numpy as np

a = list(np.random.random(1000))

rdd = sc.parallelize(a)
rdd = rdd.map(lambda x: (x,1))
rddc = rdd.cartesian(rdd)

#rddc.saveAsTextFile('/content/drive/My Drive/Coursework/CS 5683_Big_Data/Project-1/rddc_1000.txt')

In [7]:
f_input = "/content/drive/My Drive/Coursework/CS 5683_Big_Data/Project-1/browsing_test.dat"
#f_input = "/content/drive/My Drive/Coursework/CS 5683_Big_Data/Project-1/browsing.dat"

# read the raw data
data = sc.textFile(f_input)
# count the total number of samples
n_samples = data.count()
print(n_samples)
# min_sup to frequency
sup = 2


6


In [8]:
c1 = data.flatMap(lambda line: line.strip().split(' ')).distinct()
c1 = c1.filter(lambda x: x)

#c1c = c1.cartesian(c1).filter(lambda line : line[0] > line[1])

print(c1.take(3))
#print(c1c.count())

#for val in c1c.collect():
#  print(val)

['FRO11987', 'SNA90258', 'ELE91550']


In [9]:
## split sort
itemset = data.map(lambda line: ([item for item in line.strip().split(' ')]))

# share the whole itemset with all workers
#shared_itemset = sc.broadcast(itemset.map(lambda x: set(x)).collect())
#shared_itemset = sc.broadcast(itemset.map(lambda x: tuple(x)).collect())
shared_itemset = sc.broadcast(itemset.map(lambda x: frozenset(x)).collect())

#print('shared itemset = ', shared_itemset.value)

itemset = itemset.map(lambda x: tuple(x)).filter(lambda x: x[0])



In [10]:
def get_sup(x):
  x_sup = 0
  for t in shared_itemset.value:
    if x.issubset(t):
    #if set({x}).issubset(set(t)):
      x_sup +=1
  if x_sup >= sup:
      return x, x_sup
  else:
      return ()

def merge_list(x):
  l = []
  for item in x:
    for i in item:
      l.append(i)
  
  return set(l)

def remove_less(x,k):
  if len(x) < k:
    return ()
  else:
    return x

In [11]:
# store for all freq_k
frequent_itemset = []

c1 = data.flatMap(lambda line: line.strip().split(' ')).distinct()
c1 = c1.map(lambda x : frozenset({x}))

print(c1.take(3))

f1 = c1.map(get_sup).filter(lambda x: x) # compute frequent items excedding support threshold

print(f1.take(3))

#f1.saveAsTextFile('/content/drive/My Drive/Coursework/CS 5683_Big_Data/Project-1/f1.txt')

#frequent_itemset.append(f1.collect()) # store frequent items with support/count as value


[frozenset({'FRO11987'}), frozenset({'SNA90258'}), frozenset({'ELE91550'})]
[(frozenset({'FRO86643'}), 3), (frozenset({'ELE17451'}), 5), (frozenset({'GRO99222'}), 2)]


In [ ]:
print(f1.count())

735


In [15]:
f1m = f1.map(lambda x: tuple(x[0]))
#f1m = f1m.map(lambda x: int(x[3:]))
print(f1m.take(3))
print(f1m.count())
for val in f1m.collect():
  print(val)

[('FRO86643',), ('ELE17451',), ('GRO99222',)]
4
('FRO86643',)
('ELE17451',)
('GRO99222',)
('SNA11465',)


In [33]:
#f1c = f1m.cartesian(f1m).filter(lambda line: line[0] > line[1]).map(lambda x: frozenset(x[0]+x[1]))
f1c = f1m.cartesian(f1m).map(lambda x: frozenset(x[0]+x[1])).filter(lambda x: len(x)>1).distinct()
print(f1c.take(3))
print(f1c.count())

for val in f1c.collect():
  print(val)


[frozenset({'FRO86643', 'ELE17451'}), frozenset({'SNA11465', 'FRO86643'}), frozenset({'SNA11465', 'ELE17451'})]
6
frozenset({'FRO86643', 'ELE17451'})
frozenset({'SNA11465', 'FRO86643'})
frozenset({'SNA11465', 'ELE17451'})
frozenset({'FRO86643', 'GRO99222'})
frozenset({'ELE17451', 'GRO99222'})
frozenset({'SNA11465', 'GRO99222'})


In [18]:
f2 = f1c.map(get_sup).filter(lambda x: x) # compute frequent items excedding support threshold

print(f2.take(3))
print(f2.count())

[(frozenset({'FRO86643', 'ELE17451'}), 3), (frozenset({'SNA11465', 'FRO86643'}), 2), (frozenset({'ELE17451', 'GRO99222'}), 2)]
4


In [30]:
f2m = f2.map(lambda x: tuple(x[0]))
f2c = f2m.cartesian(f1m).map(lambda x: frozenset(x[0]+x[1])).filter(lambda x: len(x)>2).distinct()
print(f2c.take(3))

for val in f2c.collect():
  print(val)

[frozenset({'SNA11465', 'FRO86643', 'ELE17451'}), frozenset({'ELE17451', 'FRO86643', 'GRO99222'}), frozenset({'SNA11465', 'FRO86643', 'GRO99222'})]
frozenset({'SNA11465', 'FRO86643', 'ELE17451'})
frozenset({'ELE17451', 'FRO86643', 'GRO99222'})
frozenset({'SNA11465', 'FRO86643', 'GRO99222'})
frozenset({'ELE17451', 'SNA11465', 'GRO99222'})


In [24]:
a = (('FRO86643', 'ELE17451'), ('FRO86643',))
a[0] + a[1]

('FRO86643', 'ELE17451', 'FRO86643')

In [ ]:
f1m = f1.map(lambda x: x[0])

print(f1m.take(3))
print(f1m.count())

#for val in f1m.collect():
#  print(val)

[frozenset({'FRO11987'}), frozenset({'SNA90258'}), frozenset({'ELE52966'})]


KeyboardInterrupt: ignored

In [ ]:
f1c = f1m.cartesian(f1m).filter(lambda line: line[0] > line[1])
print(f1c.take(3))
print(f1c.count())

In [ ]:
aa = (({'2','1'},),'b')
frozenset(tuple(aa[0][0]) + (aa[1],))

In [ ]:
f1m = f1.map(lambda x: x[0])

#f1c = f1m.cartesian(f1m).filter(lambda line: line[0] > line[1])

print(f1m.count())
#f2p = f1c.cartesian(f1m) #.filter(lambda line: line[0] > line[1])

#for val in f2p.collect():
#  print(val)


In [ ]:
a = ('a','b','c')
b = ('a','b','c','d')

set(a).issubset(set(b))

#for item in a:
#  print(item)

In [ ]:
frequent_items = f1.map(lambda x : tuple(x)[0])
for val in frequent_items.collect():
  print(val)


In [ ]:
frequent_items = f1.map(lambda x : tuple(x[0])) # frequent items

prev = f1.map(lambda x: tuple((x[0]))) 

print('start')
fk_cartesian = prev.cartesian(frequent_items).filter(lambda line: line[0] > line[1]).map(lambda line: set((line[0][0], line[1][0])))

print(fk_cartesian.count())
#for val in fk_cartesian.collect():
#  print(val)

#fk_cartesian.saveAsTextFile('/content/drive/My Drive/Coursework/CS 5683_Big_Data/Project-1/f2.txt')

#fk_cartesian = fk_cartesian.map(merge_list)

In [ ]:
f2 = fk_cartesian.map(get_sup).filter(lambda x: x)
for val in f2.collect():
  print(val)

In [ ]:
frequent_pairs = f2.map(lambda x : tuple(x[0])) # frequent items

prev = f2.map(lambda x: tuple((x[0]))) 

fk_cartesian = prev.cartesian(frequent_items).map(lambda a : set((a[0][0],a[0][1],a[1][0]))).map(lambda x: tuple(x)).filter(lambda line: len(line) > 2).distinct()

for val in fk_cartesian.collect():
  print(val)


In [ ]:
fk_cartesian = prev.cartesian(frequent_items)
  
#fk_cartesian = fk_cartesian.map(merge_list)
fk_cartesian = fk_cartesian.map(lambda x : set([i for i in x]))

#fk_cartesian = fk_cartesian.map(lambda x : remove_less(x,3)).map(lambda x: tuple(x)).filter(lambda x: x).distinct().map(lambda x: set(x))
#fk_cartesian = fk_cartesian.map(lambda x: tuple(x)).filter(lambda line: len(line) > 2).distinct().map(lambda x: set(x))

for val in fk_cartesian.collect():
  print(val)


{('FRO86643',), ('FRO86643', 'ELE17451')}
{('ELE17451',), ('FRO86643', 'ELE17451')}
{('FRO86643', 'ELE17451'), ('GRO99222',)}
{('SNA11465',), ('FRO86643', 'ELE17451')}
{('FRO86643',), ('SNA11465', 'FRO86643')}
{('ELE17451',), ('SNA11465', 'FRO86643')}
{('GRO99222',), ('SNA11465', 'FRO86643')}
{('SNA11465',), ('SNA11465', 'FRO86643')}
{('FRO86643',), ('GRO99222', 'ELE17451')}
{('FRO86643',), ('SNA11465', 'ELE17451')}
{('ELE17451',), ('GRO99222', 'ELE17451')}
{('GRO99222',), ('GRO99222', 'ELE17451')}
{('SNA11465',), ('GRO99222', 'ELE17451')}
{('ELE17451',), ('SNA11465', 'ELE17451')}
{('SNA11465', 'ELE17451'), ('GRO99222',)}
{('SNA11465',), ('SNA11465', 'ELE17451')}


In [ ]:
{'ELE52966', 'FRO11987'} in ({'ELE52966', 'FRO11987'})

False

In [ ]:
prev.cartesian(frequent_items).filter(lambda line: line[1] not in line[0])
a = (('FRO86643',), ('ELE17451',))
a[1][0]
set((a[0],a[1]))

{('ELE17451',), ('FRO86643',)}

In [ ]:
frequent_itemset.append(fk_cartesian.collect())

In [ ]:
#print(frequent_itemset.shape())
f1.saveAsTextFile('/content/drive/My Drive/Coursework/CS 5683_Big_Data/Project-1/f1.txt')

In [ ]:
def find_frequent_itemset(prev, frequent_items, sup, k, frequent_itemset):
  print(k)
  fk_cartesian = prev.cartesian(frequent_items)
  
  fk_cartesian = fk_cartesian.map(merge_list)

  fk_cartesian = fk_cartesian.map(lambda x : remove_less(x,k)).map(lambda x: tuple(x)).filter(lambda x: x).distinct().map(lambda x: set(x))

  next = fk_cartesian.map(get_sup).filter(lambda x: x)

  frequent_itemset.append(next.collect())

  print(k, ' ', len(frequent_itemset[k-1]))

  prev = next.map(lambda x: tuple(x[0]))

  k = k+1

  if k == 3:    
    return 

  find_frequent_itemset(prev, frequent_items, sup, k, frequent_itemset)

#f1 = c1.map(get_sup).filter(lambda x: x) # compute frequent items excedding support threshold

#frequent_itemset.append(f1.collect()) # store frequent items with support/count as value

frequent_items = f1.map(lambda x : tuple(x[0])) # frequent items

prev = f1.map(lambda x: tuple(x[0])) # 

find_frequent_itemset(prev, frequent_items, sup, 2,frequent_itemset)


In [ ]:
fall = sc.parallelize(frequent_itemset)
fall = fall.flatMap(lambda x: x)


In [ ]:
def separate_items(x,k):
  if len(x[0]) == k:
    return x

fk_1 = fall.map(lambda x: separate_items(x,1)).filter(lambda x: x)
fk_2 = fall.map(lambda x: separate_items(x,2)).filter(lambda x: x)
fk_3 = fall.map(lambda x: separate_items(x,3)).filter(lambda x: x)

print('k = 1')
for val in fk_1.collect():
  print(val)

print('k = 2')
for val in fk_2.collect():
  print(val)

print('k = 3')
for val in fk_3.collect():
  print(val)


k = 1
({'FRO86643'}, 3)
({'ELE17451'}, 5)
({'GRO99222'}, 2)
({'SNA11465'}, 3)
k = 2
({'GRO99222', 'ELE17451'}, 2)
({'FRO86643', 'ELE17451'}, 3)
({'FRO86643', 'SNA11465'}, 2)
({'ELE17451', 'SNA11465'}, 3)
k = 3
({'FRO86643', 'ELE17451', 'SNA11465'}, 2)


In [ ]:
x = ({'GRO99222', 'ELE17451'}, 2)
if len(x[0]) == 2:
  print(x)

({'GRO99222', 'ELE17451'}, 2)


In [ ]:
fkm1_no_value = f1.map(lambda x: tuple(x[0]))
fkm1_cartesian = fkm1_no_value.cartesian(fkm1_no_value)

def merge_list(x):
  l = []
  for item in x:
    for i in item:
      l.append(i)
  
  return set(l)

def remove_less(x,k):
  if len(x) < k:
    return ()
  else:
    return x

fkm1_cartesian = fkm1_cartesian.map(merge_list)

fkm1_cartesian = fkm1_cartesian.map(lambda x : remove_less(x,2)).map(lambda x: tuple(x)).filter(lambda x: x).distinct().map(lambda x: set(x))

fkm2 = fkm1_cartesian.map(get_sup).filter(lambda x: x)

for val in fkm2.collect():
  print(val)



({'GRO99222', 'ELE17451'}, 2)
({'ELE17451', 'FRO86643'}, 3)
({'SNA11465', 'FRO86643'}, 2)
({'SNA11465', 'ELE17451'}, 3)


In [ ]:
fjoin = f1.union(fkm2)
for val in fjoin.collect():
  print(val)

({'FRO86643'}, 3)
({'ELE17451'}, 5)
({'GRO99222'}, 2)
({'SNA11465'}, 3)
({'GRO99222', 'ELE17451'}, 2)
({'ELE17451', 'FRO86643'}, 3)
({'SNA11465', 'FRO86643'}, 2)
({'SNA11465', 'ELE17451'}, 3)


In [ ]:
fkm2_no_value = fkm2.map(lambda x: tuple(x[0]))

fkm2_cartesian = fkm2_no_value.cartesian(fkm1_no_value)

fkm2_cartesian = fkm2_cartesian.map(merge_list)

fkm2_cartesian = fkm2_cartesian.map(lambda x : remove_less(x,3)).map(lambda x: tuple(x)).filter(lambda x: x).distinct().map(lambda x: set(x))

fkm3 = fkm2_cartesian.map(get_sup).filter(lambda x: x)

for val in fkm3.collect():
  print(val)


({'SNA11465', 'ELE17451', 'FRO86643'}, 2)


In [ ]:
from operator import add

fkm_nv = fkm.map(lambda x: list(x[0]))

fkm_nv = fkm_nv.cartesian(fkm_nv)

def merge_list(x):
  l = []
  for item in x:
    l.append(item[0])
  
  return l

fkm_nv = fkm_nv.map(merge_list)

fkm_nv = fkm_nv.map(lambda x: (x,0))

fkm_nv = fkm_nv.map(lambda x: (tuple(x[0]),x[1])).reduceByKey(add) 

fkm_nv = fkm_nv.map(lambda x: set(x[0]))

def remove_less(x,k):
  if len(x) < k:
    return ()
  else:
    return x

fkm_nv = fkm_nv.map(lambda x : remove_less(x,2)).map(lambda x: tuple(x)).distinct().filter(lambda x: x).map(lambda x: set(x))

for val in fkm_nv.collect():
  print(val)

fkm_nv = fkm_nv.map(get_sup).filter(lambda x: x)

for val in fkm_nv.collect():
  print(val)


{'GRO99222', 'ELE17451'}
{'ELE17451', 'FRO86643'}
{'GRO99222', 'FRO86643'}
{'SNA11465', 'FRO86643'}
{'SNA11465', 'ELE17451'}
{'SNA11465', 'GRO99222'}
({'GRO99222', 'ELE17451'}, 2)
({'ELE17451', 'FRO86643'}, 3)
({'SNA11465', 'FRO86643'}, 2)
({'SNA11465', 'ELE17451'}, 3)


In [ ]:
from operator import add
rdd = sc.parallelize([({"a","b"}, 1), ({"b","a"}, 2)])
sorted(rdd.map(lambda x: (tuple(x[0]),x[1])).reduceByKey(add).collect())

[(('a', 'b'), 3)]

In [ ]:
l = {1}
l.add(3)
#for i in list(['FRO86643'], ['FRO86643']):
#  print(i)

len({1})
#print(list({'FRO86643','ELE17451'}))

1

In [ ]:
f_input = "/content/drive/My Drive/Coursework/CS 5683_Big_Data/Project-1/test.dat"

# read the raw data
data = sc.textFile(f_input)
# count the total number of samples
n_samples = data.count()
# min_sup to frequency
sup = n_samples * 0.5

# split sort
itemset = data.map(lambda line: sorted([int(item) for item in line.strip().split(' ')]))

# share the whole itemset with all workers
#shared_itemset = sc.broadcast(itemset.map(lambda x: set(x)).collect())

shared_itemset = sc.broadcast(itemset.map(lambda x: set(x)).collect())

print('shared itemset = ', shared_itemset.value)

# store for all freq_k
frequent_itemset = []

# prepare candidate_1
k = 1

c_k = itemset.flatMap(lambda x: set(x)).distinct() #.collect()
c_k1 = itemset.flatMap(lambda x: set(x)).distinct().collect()
c_k1 = [{x} for x in c_k1]

def get_sup(x):
  x_sup = 0
  for t in shared_itemset.value:
    if x.issubset(t):
      x_sup +=1
  if x_sup >= sup:
      return x, x_sup
  else:
      return ()

c_kr = data.flatMap(lambda line: line.split(" ")).distinct() #itemset.flatMap(lambda x: x)
c_kr = c_kr.map(lambda x: set([int(x)]))

fkm = c_kr.map(get_sup).filter(lambda x: x)

frequent_itemset.append(fkm.collect())
#frequent_itemset.append(fkm.collect())
print('frequent item = ', fkm.collect())

print('next candidate = ', fkm.map(lambda x: x[0]).collect())

def combinations(row):
  l = row[0]
  return [(k,0) for k in itertools.combinations(l, 2)]

next_ck_nc = fkm.flatMap(lambda x: x[0])

def remove(x):
  if x[0] < x[1]:
    return [x[0],x[1]]

def mergek(x):
  l = []
  for item in list(x[0]):
    if item < x[1]:
      l.append(item)
    else:
      return ()
  l.append(x[1])
  return l

def merge(x):
  l = []
  for item in list([x[0]]):
    if item < x[1]:
      l.append(item)
    else:
      return ()
  l.append(x[1])
  return l


next_ck_nc = next_ck_nc.cartesian(next_ck_nc).filter(remove)
next_ck_nc = next_ck_nc.map(lambda x: set(x))
#next_ck_nc = next_ck_nc.map(get_sup).filter(lambda x: x)
print(next_ck_nc.collect())

fkm0 = fkm.flatMap(lambda x: x[0])
#next_ck_nc1 = 
next_ck_nc2 = next_ck_nc.cartesian(fkm0) #.filter(remove)
print(next_ck_nc2.collect())
next_ck_nc2 = next_ck_nc2.map(mergek)
next_ck_nc2 = next_ck_nc2.filter(lambda x: x)
next_ck_nc2 = next_ck_nc2.map(lambda x: set(x))
print(next_ck_nc2.collect())

next_ck_nc1 = fkm0.cartesian(fkm0)
next_ck_nc1 = next_ck_nc1.map(merge)
next_ck_nc1 = next_ck_nc1.filter(lambda x: x)
next_ck_nc1 = next_ck_nc1.map(lambda x: set(x))
print(next_ck_nc1.collect())

#next_ck_nc.map(combinations).flatMap(lambda x: x).collect()
#next_ck_nc = next_ck_nc.map(combinations).flatMap(lambda x: x).collect()


next_ck = fkm.map(lambda x: x[0]).collect()
print(next_ck)
next_c = [var1 | var2 for index, var1 in enumerate(next_ck) for var2 in next_ck[index + 1:] if
              list(var1)[:k - 2] == list(var2)[:k - 2]]

next_c_kr = sc.parallelize(next_c).map(get_sup).filter(lambda x: x)
#next_ck = [item for item in map(lambda x: x[0], fkm.collect())]
print(next_c_kr.collect())

next_ck_nc = next_c_kr.map(lambda x: x[0])
print(next_ck_nc.collect())



shared itemset =  [{1, 2, 5}, {2, 3, 4}, {3, 4}, {1, 2, 3, 4}]
frequent item =  [({1}, 2), ({4}, 3), ({2}, 3), ({3}, 3)]
next candidate =  [{1}, {4}, {2}, {3}]
[{1, 4}, {1, 2}, {1, 3}, {2, 4}, {3, 4}, {2, 3}]
[({1, 4}, 1), ({1, 4}, 4), ({1, 4}, 2), ({1, 4}, 3), ({1, 2}, 1), ({1, 2}, 4), ({1, 3}, 1), ({1, 3}, 4), ({1, 2}, 2), ({1, 2}, 3), ({1, 3}, 2), ({1, 3}, 3), ({2, 4}, 1), ({2, 4}, 4), ({3, 4}, 1), ({3, 4}, 4), ({2, 4}, 2), ({2, 4}, 3), ({3, 4}, 2), ({3, 4}, 3), ({2, 3}, 1), ({2, 3}, 4), ({2, 3}, 2), ({2, 3}, 3)]
[{1, 2, 4}, {1, 3, 4}, {1, 2, 3}, {2, 3, 4}]
[{1, 4}, {1, 2}, {1, 3}, {2, 4}, {3, 4}, {2, 3}]
[{1}, {4}, {2}, {3}]
[({1, 2}, 2), ({2, 4}, 2), ({3, 4}, 3), ({2, 3}, 2)]
[{1, 2}, {2, 4}, {3, 4}, {2, 3}]


In [ ]:
l = []
l1 = ({1, 4}, 5)
print(list([l1[0]]))

for item in list(l1[0]):
  print(item)
  if item < l1[1]:
    l.append(item)
l.append(l1[1])
print(l)

[{1, 4}]
1
4
[1, 4, 5]


In [ ]:
import itertools
a = [k for k in itertools.combinations([(1,2),(3,4)], 2)]
print(a)

[((1, 2), (3, 4))]


In [ ]:
a = {2}
#a.issubset([1,2,5])
[1,2] in [1,2,5]

False

In [ ]:
a = set([1,1])
b = set([(1,2)])
print(a,b)
sorted((2,1))

#a[0]

{1} {(1, 2)}


[1, 2]

In [ ]:
a = set(1)
b = set([1,23])
print(a)
a.issubset(b)

TypeError: ignored

In [ ]:
a = sc.parallelize([
    ((1,2),1),
    ((2,1),2),
    ((3,4),3)
  ])
a = a.map(lambda x: (tuple(sorted(x[0])),x[1]))
a = a.reduceByKey(lambda a,b: a+b)
a = a.map(lambda x: (set(x[0]),x[1]))
print(a.collect())

[({1, 2}, 3), ({3, 4}, 3)]


###### Import required libraries now

In [ ]:
import os
import sys
from pyspark import SparkContext, SparkConf

###### Let's initialize Spark context now

In [ ]:
# create Spark context with necessary configuration
sc = SparkContext("local","PySpark Word Count Exmaple")

###### Follow the tutorial to mount your Google Drive. Give mounted Drive paths below

In [ ]:
# Give **.txt FILE PATHS** here
file1 = '/content/drive/My Drive/file1.txt' # change the path  
file2 = '/content/drive/My Drive/file2.txt' # change the path  

filer = open('/content/drive/My Drive/file1.txt', 'r') 
Lines = filer.readlines()

count = 0
# Strips the newline character 
for line in Lines: 
    print("Line{}: {}".format(count, line.strip())) 


# USE THESE FILES as input(s) FOR ALL BELOW QUESTIONS

Line0: my name is Suraj
Line0: my name is Suraj
Line0: my name is Suraj
Line0: my name is Suraj1


### Example Spark program

In [ ]:
# Example Spark application for a simple wordcount
# What is wordcount? 
    # Given a file, count the frequency of all words appearing in that file
    
# Step-1: Read the required file. In our case it is file1 or file2.
# NOTE: We do not need to initialize SparkContext as only one SparkContext can be initialized in one notebook
fileRDD = sc.textFile(file1)

# Step-2: 
    # Each line in our file(s) is a sentence. So, we need to split the sentence with ' ' to get words
    # Using map() will return RDD[list]. But we need RDD[string]. So we use flatMap()
wordsRDD = fileRDD.flatMap(lambda line: line.split(" ")) # <----------- TEST what happens when you use map()

wordsRDDm = fileRDD.map(lambda line: line.split(" ")) # <----------- TEST what happens when you use map()

# Step-3: For each input, we will make (K,V) pair, where K is the word and V is 1
pairRDD = wordsRDDm.map(lambda word: (word,1))

# Step-4: Now we have to sum all 1's of each word
# NOTE: A word may present in multiple data partitions. So we use reduceByKey() to group by key and perform sum
countRDD = pairRDD.reduceByKey(lambda a,b: a+b)

#Step-5: Save results in a text file
#pairRDD.saveAsTextFile('/content/drive/My Drive/Coursework/CS 5683_Big_Data/file13.txt') # <----------- GIVE FILE PATH
countRDD.saveAsTextFile('/content/drive/My Drive/file10.txt') # <----------- GIVE FILE PATH

### Question - 1 (10 points) 

Write a Spark application that counts number of words that ends with each letter in file1. That is for each letter count the number of (non-unique) words that ends with the specific letter. You can **ignore** letter cases (consider the given text contains only lower-case letters). Also, you can **ignore** words that end with non-alphabetic letters. **Sort the output in alphabetical order**

> Indented block



Example Output:
('a', 500)
('B', 100)

which means that the given input has 500 words that end with the letter 'a' and 100 words that end with letter 'B'.
NOTE: The output is sorted, the application counts duplicate words also, and the application is not case-sensitive

In [ ]:
# YOUR CODE for Question-1 HERE

In [ ]:
fileRDD = sc.textFile(file1)

wordsRDD = fileRDD.flatMap(lambda line: line.split(" ")) # <-------------------- TEST what happens when you use map()

wordsRDDf = wordsRDD.filter(lambda word: word.lower()[-1].isdigit() == False) # remove words that ends with digits

pairRDD = wordsRDDf.map(lambda word: (word[-1],1)) # map with key = last letter of word, and value = 1

countRDD = pairRDD.reduceByKey(lambda a,b: a+b) # reduce keys with summation operation

countRDDsort = countRDD.sortByKey(ascending=True) # sort using keys in ascending order

countRDDsort.saveAsTextFile('/content/drive/My Drive/file_sol_q1.txt') # <----------- GIVE FILE PATH


In [ ]:
# PRINT THE OUTPUT HERE
for x in countRDDsort.collect():
  print(x)

('J', 1)
('e', 4)
('j', 2)
('s', 4)
('y', 4)


### Question - 2 (10 points)

Write a Spark application that counts the **unique** number of words that have **n-letters** in file1. That is count the number of **unique** 1-letter words, 2-letter words, 3-letter words, etc. You can **include** all non-alphabetic letters for this application.

Example Output:
(1, 100)
(2, 700)
(3, 1500)

which means that the input file for the Spark application has **100 unique 1-letter words** , **700 unique 2-letter words** , and **1500 uniqe 3-letter words**

In [ ]:
# YOUR CODE for Question-2 HERE

In [ ]:
fileRDD = sc.textFile(file1)

wordsRDD = fileRDD.flatMap(lambda line: line.split(" ")) # <----------- TEST what happens when you use map()

distinctwordsRDD = wordsRDD.distinct() # remove duplicate entries

pairRDD = distinctwordsRDD.map(lambda word: (len(word),1)) # map with key = length of word, and value = 1

countRDD = pairRDD.reduceByKey(lambda a,b: a+b) # reduce keys with summation operation

countRDDsort = countRDD.sortByKey(ascending=True) # sort using keys in ascending order

countRDDsort.saveAsTextFile('/content/drive/My Drive/file_sol_q2.txt') # <----------- GIVE FILE PATH

In [ ]:
# PRINT THE OUTPUT HERE
for x in countRDDsort.collect():
  print(x)

### Question - 3 (10 points)

Write a Spark aplication that outputs wordcount from two files: file1 and file2. That is count the number of occurances of words from two files. You can **ignore** letter cases (consider the given input files contain only lower-case letters). Also, you can **ignore** that is not present in both files.

Example Output:
(big, (10, 20))
(Data, (30, 50))

which means the word "big" appears 10 times in file-1 and 20 times in file-2 and the word "Data" appears 30 times in file-1 and 50 times in file-2

In [ ]:
# YOUR CODE for Question-3 HERE

In [ ]:
fileRDD1 = sc.textFile(file1)
fileRDD2 = sc.textFile(file2)

wordsRDD1 = fileRDD1.flatMap(lambda line: line.split(" ")) # <----------- TEST what happens when you use map()
wordsRDD2 = fileRDD2.flatMap(lambda line: line.split(" ")) # <----------- TEST what happens when you use map()

pairRDD1 = wordsRDD1.map(lambda word: (word,1)) # map with key = word, and value = 1
pairRDD2 = wordsRDD2.map(lambda word: (word,1)) # map with key = word, and value = 1

countRDD1 = pairRDD1.reduceByKey(lambda a,b: a+b) # reduce keys with summation operation
countRDD2 = pairRDD2.reduceByKey(lambda a,b: a+b) # reduce keys with summation operation

countRDD3 = countRDD1.join(countRDD2).reduceByKey(lambda x,y : (x,y)) # join operation
#countRDD3 = countRDD1.union(countRDD2).reduceByKey(lambda x,y : (x,y)) # union operation

countRDD3.saveAsTextFile('/content/drive/My Drive/file_sol_q3.txt') # <----------- GIVE FILE PATH

In [ ]:
# PRINT THE OUTPUT HERE
for x in countRDD3.collect():
  print(x)

### WHAT TO TURN-IN IN CANVAS

1. Complete questions 1,2, and 3
2. Download this completed python notebook as .ipynb
3. Upload it in Canvas assignment

# Due Date: Sept. 3 at 11:59pm